# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/bitcoin/4_bitcoin_batch_inference.ipynb)

## 🗒️ This notebook is divided into the following sections:

1. Use the deployment to make a prediciton.
2. Retrieve batch data.
3. Batch prediction using deployment.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib
import time

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
feature_view = fs.get_feature_view(
    name='fv_name',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🪝 Retrieve Deployment</span>


In [ ]:
# get deployment object
deployment = ms.get_deployment("btcmodeldeployment")

To start it you need to run:

In [ ]:
deployment.start(await_running=120)

---
## <span style="color:#ff5f27;">🔮 Predicting</span>

Using the deployment let's use the input example that we registered together with the model to query the deployment.

In [ ]:
today = datetime.today()
latest_date = int(time.mktime(today.timetuple()) * 1000) # converting todays datetime to unix

deployment_output = deployment.predict(inputs=[latest_date])
# or deployment.predict({ "instances": [[latest_date]] })

deployment_output

In [ ]:
pred_encoded = pd.DataFrame(
    deployment_output["predictions"],
    columns=["close"],
) # since we applied transformation function to the 'close' columns,
  # now we need to provide a df with the same column name to decode.
                                                
pred = decode_features(pred_encoded, feature_view=feature_view)

In [ ]:
pred = pred.rename(columns={"close": "predicted_price"})
pred["datetime"] = [today.strftime("%Y-%m-%d")]
pred

## <span style="color:#ff5f27;"> ✨ Load Batch Data of last days</span>

In [ ]:
feature_view.init_batch_scoring(training_dataset_version=1)

batch_data = feature_view.get_batch_data().drop('unix',axis=1)
batch_data.head()

In [ ]:
model = mr.get_model("bitcoin_price_model", version=1)
model_dir = model.download()

loaded_model = tf.saved_model.load(model_dir)
serving_function = loaded_model.signatures["serving_default"]

In [ ]:
predictions_batch = serving_function(tf.constant(
    batch_data.values.reshape(
        -1,batch_data.shape[0],batch_data.shape[1]),
    tf.float32
))['dense_1'].numpy()

Recall that you applied transformation functions, such as min max scaler and laber encoder. 

Now you want to transform them back to human readable format.

In [ ]:
pred_batch_encoded = pd.DataFrame(
    predictions_batch,
    columns=["close"],
) # since we applied transformation function to the 'close' columns,
  # now we need to provide a df with the same column name to decode.
                                                
pred_batch = decode_features(pred_batch_encoded, feature_view=feature_view)
pred_batch.tail(5)

Stop your deployment

In [ ]:
deployment.stop(await_stopped=180)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Bitcoin price prediction tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai